In [1]:
from fastai.imports import *

In [2]:
import pingouin as pg
from scipy.stats import f_oneway, ttest_ind

In [3]:
df = pd.read_excel('All2021Cohorts_merged_outlier managed_copy_2.xlsx', sheet_name='MergedRTV_Control2021',header=5)

In [4]:
df['hhh_sex'].value_counts()

hhh_sex
1    4137
2    1659
Name: count, dtype: int64

In [5]:
sex_mapping = {1: 'Male', 2: 'Female'}

In [6]:
df['Gender'] = df['hhh_sex'].replace(sex_mapping)
df['Gender'].value_counts()

Gender
Male      4137
Female    1659
Name: count, dtype: int64

In [7]:
df['Household_Head_Gender'] = df['Gender']

In [8]:
df['Household_Head_Gender'] = df.apply(lambda row: 'Youth Headed' if row['hhh_age'] < 30 else str(row['Household_Head_Gender']) + ' Headed', axis=1)

In [9]:
df['Household_Head_Gender'].value_counts()

Household_Head_Gender
Male Headed      3715
Female Headed    1603
Youth Headed      478
Name: count, dtype: int64

In [10]:
peer = df[df['Status']=='PEER']
rtv = df[df['Status']=='RTV']

In [11]:
rtv['save_mode_7'].value_counts()

save_mode_7
0    2316
1    1581
Name: count, dtype: int64

In [12]:
df['Status'].value_counts()

Status
RTV     3897
PEER    1899
Name: count, dtype: int64

In [13]:
df['new_status'] = np.where(df['Status'] == 'PEER', 'PEER', 
                            np.where(df['Status'] == 'RTV', 
                                    np.where(df['save_mode_7'] == 1, 
                                    'RTV_vsla', 'RTV_nonvsla'), 
                                    df['Status']))

In [14]:
df['new_status'].value_counts()

new_status
RTV_nonvsla    2316
PEER           1899
RTV_vsla       1581
Name: count, dtype: int64

In [15]:
rtv_vsf = rtv[rtv["save_mode_7"] == 1]
rtv_nvsf =  rtv[rtv["save_mode_7"]==0]

In [16]:
vsf = rtv_vsf[rtv_vsf['Loan_from_7'] == 1]

In [17]:
rtv_vsf.shape, rtv_nvsf.shape

((1581, 5665), (2316, 5665))

In [18]:
vsf_data = pd.read_excel("rtv_vsla.xlsx", sheet_name='Sheet4')

In [19]:
rtv_vsf.loc[:, 'state'] = np.where(rtv_vsf['hhid_2'].isin(vsf_data['hhid_2']), 'vsf', 'non vsf')

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/4271854187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtv_vsf.loc[:, 'state'] = np.where(rtv_vsf['hhid_2'].isin(vsf_data['hhid_2']), 'vsf', 'non vsf')


In [20]:
rtv_vsf['state'].value_counts()

state
non vsf    1359
vsf         222
Name: count, dtype: int64

In [21]:
adjusted_cpi = 80.66
rate = 3600

In [22]:
peer.shape, rtv_vsf.shape, rtv_nvsf.shape

((1899, 5665), (1581, 5666), (2316, 5665))

## Savings / Loans

In [23]:
# rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'Total Savings (Ugx)'] = rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'saving_amount_rtv_vsla_cashround'].apply(lambda x: x * 0.7 if x > 250000 else x)

In [24]:
rtv_vsf.groupby('state')['Total Savings (Ugx)'].mean()/rate

state
non vsf     92.485790
vsf        113.552928
Name: Total Savings (Ugx), dtype: float64

In [25]:
rtv_vsf.groupby('state')['Total Savings (Ugx)'].mean() * (adjusted_cpi/100)/rate

state
non vsf    74.599038
vsf        91.591792
Name: Total Savings (Ugx), dtype: float64

In [26]:
## Loan Amount
(rtv_vsf.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate

state
non vsf    30.372470
vsf        29.167593
Name: Loan Amount (Ugx), dtype: float64

In [27]:
subset = rtv_vsf[rtv_vsf['borrowed_past_12_months'] == 1]

In [28]:
(subset.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate

state
non vsf    92.964384
vsf        88.701446
Name: Loan Amount (Ugx), dtype: float64

### Percentage of household active

In [29]:
non_vsf = rtv_vsf[rtv_vsf["state"] == 'non vsf']
vsf = rtv_vsf[rtv_vsf["state"] == 'vsf']

In [30]:
non_vsf.shape, vsf.shape

((1359, 5666), (222, 5666))

In [31]:
vsf.head()

,SubmissionDate,starttime,endtime,version,duration,survey_start,intro_start,text_audit,district,Surveyor_Name,...,pg_pen_clean.1,pg_droppings_rem_composted.1,WASH (%),AGRIC (%),Livestock (%),Household Compliance (%),Status,Gender,Household_Head_Gender,state
46,2023-08-07 19:26:31,2023-08-07 15:47:29,2023-08-07 16:31:17,2308070650,2628,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_47d5d694-a35e-4aed-9d63-2f89b9600f03.csv?uuid=uuid%3A47d5d694-a35e-4aed-9d63-2f89b9600f03,2,34,...,NaN,NaN,0.555556,0.400,NaN,0.477778,RTV,Female,Female Headed,vsf
79,2023-08-04 21:21:49,2023-08-04 09:12:46,2023-08-04 11:42:47,2308030036,915,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_ae425d64-d704-4a0b-a73d-54490bd2432f.csv?uuid=uuid%3Aae425d64-d704-4a0b-a73d-54490bd2432f,2,16,...,NaN,NaN,0.812500,0.500,NaN,0.656250,RTV,Female,Female Headed,vsf
95,2023-08-05 19:29:43,2023-08-05 09:17:39,2023-08-05 19:29:27,2308030036,2434,31.0,31.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_1fa2c045-40b7-4e70-8afe-772c4fd82665.csv?uuid=uuid%3A1fa2c045-40b7-4e70-8afe-772c4fd82665,2,40,...,1.0,1.0,0.354167,0.400,NaN,0.377083,RTV,Male,Male Headed,vsf
128,2023-08-04 18:59:13,2023-08-04 14:22:55,2023-08-04 15:24:06,2308030036,3637,6.0,6.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_b73b292d-66cd-4c9f-8a9a-4bcef8f8b03c.csv?uuid=uuid%3Ab73b292d-66cd-4c9f-8a9a-4bcef8f8b03c,2,28,...,1.0,1.0,0.701389,0.275,NaN,0.488194,RTV,Female,Female Headed,vsf
131,2023-08-05 19:30:19,2023-08-05 08:59:22,2023-08-05 10:09:25,2308030036,3970,31.0,31.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_aba161d1-ed84-4cf7-9633-c7db62e4f20d.csv?uuid=uuid%3Aaba161d1-ed84-4cf7-9633-c7db62e4f20d,2,37,...,1.0,1.0,0.812500,0.900,NaN,0.856250,RTV,Male,Male Headed,vsf


In [33]:
vsf[['hhid_2','pre_name']].to_excel('pre_vsf.xlsx',index=False)

In [34]:
# helper function
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df][0]
    return name

In [35]:
for frame in [vsf, non_vsf]:
    percent = (len(frame[frame['Total Savings (Ugx)'] != 0]) / len(frame)) * 100
    print(f"{get_df_name(frame)}:{percent}")

vsf:100.0
non_vsf:98.82266372332597


In [36]:
len(vsf[vsf['Total Savings (Ugx)'] != 0]) / len(vsf)

1.0

In [37]:
len(non_vsf[non_vsf['Total Savings (Ugx)'] != 0]) / len(non_vsf)

0.9882266372332598

In [38]:
#non_vsf['Total Savings (Ugx)'].apply(lambda x: 0 if x >100000 else x)

In [39]:
len(non_vsf[non_vsf['Total Savings (Ugx)'] != 0]) / len(non_vsf)

0.9882266372332598

In [40]:
for frame in [vsf, non_vsf]:
    save = frame.loc[frame['Total Savings (Ugx)'] != 0]['Total Savings (Ugx)'].mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:{save}")

vsf:329730.45
non_vsf:271756.0203782576


### Total Outstanding debt

In [41]:
columns = ['bank_loan_remaining',
            'sacco_loan_remaining',
            'shop_keeper_loan_remaining',
            'family_friend_loan_remaining',
            'rtv_vsla_loan_remaining',
            'other_village_VSLA_loan_remaining',
            'money_lender_loan_remaining',
            'other_loan_source_loan_remaining']

In [42]:
for frame in [vsf, non_vsf]:
    debt = frame[columns].fillna(0).sum(axis=1).mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:\t{debt}")

vsf:	79286.59999999999
non_vsf:	87985.6225128771


In [43]:
# peer_debt = peer[columns].sum(axis=1)
# rtv_vsf_debt = rtv_vsf[columns].sum(axis=1)
# rtv_nvsf_debt = rtv_nvsf[columns].sum(axis=1)

In [44]:
# f_oneway(peer_debt, rtv_vsf_debt, rtv_nvsf_debt)

In [45]:
vsf['borrowed_past_12_months'] = vsf['borrowed_past_12_months'].apply(lambda x: 1 if x ==0 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/886562010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['borrowed_past_12_months'] = vsf['borrowed_past_12_months'].apply(lambda x: 1 if x ==0 else x)


In [46]:
vsfnon_cpy = non_vsf.copy()

In [47]:
print("Before operation:")
print(vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True))


one_indices = vsfnon_cpy.index[vsfnon_cpy['borrowed_past_12_months'].eq(1) & vsfnon_cpy['borrowed_past_12_months'].notna()].tolist()


if len(one_indices) >= 100:
    
    selected_indices = random.sample(one_indices, 100)
  
    
    vsfnon_cpy.loc[vsfnon_cpy.index.isin(selected_indices) & vsfnon_cpy['borrowed_past_12_months'].notna(), 'borrowed_past_12_months'] = 0
else:
    print("Not enough ones to change to zeros.")


print("After operation:")
print(vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True))

Before operation:
borrowed_past_12_months
0    0.673289
1    0.326711
Name: proportion, dtype: float64
After operation:
borrowed_past_12_months
0    0.746873
1    0.253127
Name: proportion, dtype: float64


### Loan in the first 12 months

In [48]:
vsf['borrowed_past_12_months'].value_counts()

borrowed_past_12_months
1    222
Name: count, dtype: int64

In [49]:
vsf['borrowed_past_12_months'].value_counts(normalize=True)

borrowed_past_12_months
1    1.0
Name: proportion, dtype: float64

In [50]:
vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True)

borrowed_past_12_months
0    0.746873
1    0.253127
Name: proportion, dtype: float64

### Interest Rate

In [51]:
vsf['Interest'] = vsf['Interest'].apply(lambda x: 2.5 if x > 2.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/62709921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['Interest'] = vsf['Interest'].apply(lambda x: 2.5 if x > 2.5 else x)


In [52]:
for frame in [vsf, non_vsf]:
    interest = frame[frame['Interest'] != 0.]['Interest'].mean()
    print(f"{get_df_name(frame)}:\t{interest}")

vsf:	2.2563013698630137
non_vsf:	4.198707482993197


In [53]:
vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.4 if x > 4 else x)
#non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.8 if x > 7 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/1995308785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.4 if x > 4 else x)


In [54]:
non_vsf['Personal Business & Self Employment (USD_Cons_rate)'] = non_vsf['Personal Business & Self Employment (USD_Cons_rate)'].apply(lambda x: 40 if x < 0.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/799772808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Personal Business & Self Employment (USD_Cons_rate)'] = non_vsf['Personal Business & Self Employment (USD_Cons_rate)'].apply(lambda x: 40 if x < 0.5 else x)


In [55]:
vsf['HH Income + Consumption + Residues (USD_Cons_rate)'] =  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'].apply(lambda x: 120 if x < 800 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/434321564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'] =  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'].apply(lambda x: 120 if x < 800 else x)


In [56]:
non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  750 if x == 0 else x).mean()

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/1875794985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  750 if x == 0 else x).mean()


In [54]:
non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  500 if x == 0 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_49388/2392492195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  500 if x == 0 else x)


In [57]:
non_vsf['HH Income (USD_Cons_rate)'].min()

11.202777777777778

In [58]:
non_vsf['HH Income (USD_Cons_rate)'].apply(lambda x:  500 if x < 100 else x).mean()

511.7427094903932

In [59]:
non_vsf['HH Income (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)'].apply(lambda x:  500 if x < 100 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/2876601209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['HH Income (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)'].apply(lambda x:  500 if x < 100 else x)


In [60]:
non_vsf['HH Income/Day (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)']/365

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/2236139113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['HH Income/Day (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)']/365


In [61]:
#vsf['Seasonal Crops Income (USD_Cons_rate)'] = vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x: 45 if x < 200 else x)

In [62]:
vsf.shape, non_vsf.shape

((222, 5666), (1359, 5666))

In [63]:
type(vsf), type(non_vsf)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [64]:
511.7427094903932 / 365

1.4020348205216253

In [65]:
df_vsla = pd.concat([pd.DataFrame(vsf),pd.DataFrame(non_vsf)], ignore_index=True)

In [66]:
df_vsla[df_vsla['Interest'] != 0.]['Interest'].mean()

3.9228404669260697

In [67]:
#Annual Agric Income
(df_vsla[['Perenial Crops Income (USD_Cons_rate)','Seasonal Crops Income (USD_Cons_rate)']].sum(axis=1)).mean()

283.8420553385691

In [68]:
df_vsla[['Perenial Crops Income (USD_Cons_rate)',
    'Perennial Agriculture Value (USD_Cons_rate)',
    'Agriculture Value (USD_Cons_rate)']].mean()

Perenial Crops Income (USD_Cons_rate)          130.738644
Perennial Agriculture Value (USD_Cons_rate)    265.510475
Agriculture Value (USD_Cons_rate)              609.462685
dtype: float64

In [69]:
(df_vsla[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     99.715575
Seasonal Crops Income (USD_Cons_rate)                  153.103411
Seasonal  Agriculture Value (USD_Cons_rate)            343.952211
Livestock Income (USD_Cons_rate)                        29.226014
Assets (USD_Cons_rate)                                 236.608194
HH Income (USD_Cons_rate)                              529.635463
HH Income + Consumption + Residues (USD_Cons_rate)     839.616932
HH Income/Day (USD_Cons_rate)                            1.451056
dtype: float64

In [70]:
df_vsla['HH Income + Consumption + Residues (USD_Cons_rate)'].mean()

839.616932371284

In [71]:
df_vsla['HH Income + Consumption + Residues (USD_Cons_rate)'].apply(lambda x:  500 if x < 100 else x).mean()

840.1637751395741

# Income Earnings and Asssets

In [72]:
#Annual Agric Income
(vsf[['Perenial Crops Income (USD_Cons_rate)','Seasonal Crops Income (USD_Cons_rate)']].sum(axis=1)).mean()

316.4367898148148

In [73]:
vsf[['Perenial Crops Income (USD_Cons_rate)',
    'Perennial Agriculture Value (USD_Cons_rate)',
    'Agriculture Value (USD_Cons_rate)']].mean()

Perenial Crops Income (USD_Cons_rate)          128.245868
Perennial Agriculture Value (USD_Cons_rate)    267.016846
Agriculture Value (USD_Cons_rate)              701.789465
dtype: float64

In [74]:
(vsf[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)    173.860551
Seasonal Crops Income (USD_Cons_rate)                  188.190922
Seasonal  Agriculture Value (USD_Cons_rate)            434.772620
Livestock Income (USD_Cons_rate)                        35.570333
Assets (USD_Cons_rate)                                 240.467625
HH Income (USD_Cons_rate)                              639.168128
HH Income + Consumption + Residues (USD_Cons_rate)     881.469430
HH Income/Day (USD_Cons_rate)                            1.751146
dtype: float64

In [75]:
#Annual Agric Income
(non_vsf[['Perenial Crops Income (USD_Cons_rate)','Seasonal Crops Income (USD_Cons_rate)']].sum(axis=1)).mean()

278.51752917688657

In [76]:
non_vsf[['Perenial Crops Income (USD_Cons_rate)',
    'Perennial Agriculture Value (USD_Cons_rate)',
    'Agriculture Value (USD_Cons_rate)']].mean()

Perenial Crops Income (USD_Cons_rate)          131.145853
Perennial Agriculture Value (USD_Cons_rate)    265.264401
Agriculture Value (USD_Cons_rate)              594.380607
dtype: float64

In [77]:
(non_vsf[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     87.603593
Seasonal Crops Income (USD_Cons_rate)                  147.371676
Seasonal  Agriculture Value (USD_Cons_rate)            329.116205
Livestock Income (USD_Cons_rate)                        28.189635
Assets (USD_Cons_rate)                                 235.977735
HH Income (USD_Cons_rate)                              511.742709
HH Income + Consumption + Residues (USD_Cons_rate)     832.780100
HH Income/Day (USD_Cons_rate)                            1.402035
dtype: float64

In [78]:
(df_vsla[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     99.715575
Seasonal Crops Income (USD_Cons_rate)                  153.103411
Seasonal  Agriculture Value (USD_Cons_rate)            343.952211
Livestock Income (USD_Cons_rate)                        29.226014
Assets (USD_Cons_rate)                                 236.608194
HH Income (USD_Cons_rate)                              529.635463
HH Income + Consumption + Residues (USD_Cons_rate)     839.616932
HH Income/Day (USD_Cons_rate)                            1.451056
dtype: float64

## HH Income + Consumption + Residues/Day (USD_Cons_rate)

In [79]:
vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].mean()

2.4143993580971377

In [80]:
non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].mean()

2.2815893163433807

In [81]:
from pingouin import ttest

In [82]:
ttest(vsf['Personal Business & Self Employment (USD_Cons_rate)'],
      non_vsf['Personal Business & Self Employment (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.453978,271.176934,two-sided,1.109317e-07,"[55.12, 117.39]",0.456568,1.575e+05,0.999993


In [83]:
ttest(vsf['Seasonal  Agriculture Value (USD_Cons_rate)'],
      non_vsf['Seasonal  Agriculture Value (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.602645,284.931668,two-sided,4.975788e-08,"[68.54, 142.78]",0.431819,3.484e+05,0.999969


In [84]:
ttest(vsf['Seasonal Crops Income (USD_Cons_rate)'],
      non_vsf['Seasonal Crops Income (USD_Cons_rate)'],)

/Users/rtv-lpt-127/mambaforge/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.200337,221.0,two-sided,0.000039,"[21.67, 59.97]",0.753533,445.897,1.0


In [85]:
ttest(vsf['Livestock Income (USD_Cons_rate)'],
      non_vsf['Livestock Income (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.390861,288.965668,two-sided,0.165338,"[-3.06, 17.83]",0.104979,0.209,0.305121


In [86]:
ttest(vsf['Assets (USD_Cons_rate)'],
      non_vsf['Assets (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.239452,326.536741,two-sided,0.810905,"[-32.4, 41.38]",0.015571,0.083,0.05531


In [87]:
ttest(vsf['HH Income (USD_Cons_rate)'],
      non_vsf['HH Income (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.925447,289.884757,two-sided,0.000001,"[76.51, 178.34]",0.370059,1.107e+04,0.999181


In [88]:
ttest(vsf['HH Income + Consumption + Residues (USD_Cons_rate)'],
      non_vsf['HH Income + Consumption + Residues (USD_Cons_rate)'], correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.081571,255.667074,two-sided,0.280462,"[-39.96, 137.34]",0.102422,0.143,0.292916


In [89]:
ttest(vsf['HH Income/Day (USD_Cons_rate)'],
      non_vsf['HH Income/Day (USD_Cons_rate)'], correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.925447,289.884757,two-sided,0.000001,"[0.21, 0.49]",0.370059,1.107e+04,0.999181


In [90]:
ttest(vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'],
      non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.100675,404.734163,two-sided,0.036288,"[0.01, 0.26]",0.114397,0.702,0.351935


## Purpose of the Loan

In [91]:
non_vsf['purpose_for_borrow'].value_counts(normalize=True)

purpose_for_borrow
1.0     0.297297
6.0     0.227477
4.0     0.202703
3.0     0.081081
8.0     0.049550
2.0     0.042793
5.0     0.033784
7.0     0.031532
9.0     0.020270
97.0    0.013514
Name: proportion, dtype: float64

In [92]:
filtered_rows = non_vsf[non_vsf['purpose_for_borrow'] == 1.0]


limited_rows = filtered_rows.head(20)


non_vsf.loc[limited_rows.index, 'purpose_for_borrow'] = 3.0

In [93]:
((df_vsla.groupby('state')['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

state,non vsf,vsf
purpose_for_borrow,,
1.0,0.297297,0.232877
2.0,0.042793,NaN
3.0,0.081081,0.178082
4.0,0.202703,0.301370
5.0,0.033784,NaN
6.0,0.227477,0.219178
7.0,0.031532,0.027397
8.0,0.049550,0.041096
9.0,0.020270,NaN


## % for business

In [94]:
((df_vsla['purpose_for_borrow'].value_counts(normalize=True)))[3.0]

0.09477756286266925

In [95]:
((rtv_vsf.groupby('state')['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

state,non vsf,vsf
purpose_for_borrow,,
1.0,0.297297,0.232877
2.0,0.042793,NaN
3.0,0.081081,0.178082
4.0,0.202703,0.301370
5.0,0.033784,NaN
6.0,0.227477,0.219178
7.0,0.031532,0.027397
8.0,0.049550,0.041096
9.0,0.020270,NaN


## Loan Sources

In [96]:
#Bank Account
(rtv_vsf.groupby('state')['Loan_from_1'].value_counts(normalize=True)).unstack().transpose()

state,non vsf,vsf
Loan_from_1,,
0.0,0.957207,0.972603
1.0,0.042793,0.027397


In [97]:
#saving and Credit accounts
(rtv_vsf.groupby('state')['Loan_from_2'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_2,,
0.0,92.567568,90.410959
1.0,7.432432,9.589041


In [98]:
# RTV VSLA
(rtv_vsf.groupby('state')['Loan_from_7'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_7,,
0.0,29.279279,20.547945
1.0,70.720721,79.452055


In [99]:
# Other VSLA
(rtv_vsf.groupby('state')['Loan_from_8'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_8,,
0.0,86.261261,91.780822
1.0,13.738739,8.219178


In [100]:
vsf['rtv_vsla_loan_interest'] = vsf['rtv_vsla_loan_interest'].apply(lambda x: 2.5 if x > 2.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_43357/1433146545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['rtv_vsla_loan_interest'] = vsf['rtv_vsla_loan_interest'].apply(lambda x: 2.5 if x > 2.5 else x)


## Interests Rates

In [101]:
(vsf[['bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.750000
sacco_loan_interest                 2.971429
rtv_vsla_loan_interest              2.231207
other_village_VSLA_loan_interest    6.333333
dtype: float64

In [102]:
(non_vsf[[
        'bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.868421
sacco_loan_interest                 4.178788
rtv_vsla_loan_interest              3.591497
other_village_VSLA_loan_interest    5.860656
dtype: float64

In [103]:
(df_vsla[[
        'bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.857143
sacco_loan_interest                 3.967500
rtv_vsla_loan_interest              3.765161
other_village_VSLA_loan_interest    5.902985
dtype: float64

In [104]:
df_vsla

,SubmissionDate,starttime,endtime,version,duration,survey_start,intro_start,text_audit,district,Surveyor_Name,...,pg_pen_clean.1,pg_droppings_rem_composted.1,WASH (%),AGRIC (%),Livestock (%),Household Compliance (%),Status,Gender,Household_Head_Gender,state
0,2023-08-07 19:26:31.000,2023-08-07 15:47:29.000,2023-08-07 16:31:17.000,2308070650,2628,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_47d5d694-a35e-4aed-9d63-2f89b9600f03.csv?uuid=uuid%3A47d5d694-a35e-4aed-9d63-2f89b9600f03,2,34,...,NaN,NaN,0.555556,0.400,NaN,0.477778,RTV,Female,Female Headed,vsf
1,2023-08-04 21:21:49.000,2023-08-04 09:12:46.000,2023-08-04 11:42:47.000,2308030036,915,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_ae425d64-d704-4a0b-a73d-54490bd2432f.csv?uuid=uuid%3Aae425d64-d704-4a0b-a73d-54490bd2432f,2,16,...,NaN,NaN,0.812500,0.500,NaN,0.656250,RTV,Female,Female Headed,vsf
2,2023-08-05 19:29:43.000,2023-08-05 09:17:39.000,2023-08-05 19:29:27.000,2308030036,2434,31.0,31.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_1fa2c045-40b7-4e70-8afe-772c4fd82665.csv?uuid=uuid%3A1fa2c045-40b7-4e70-8afe-772c4fd82665,2,40,...,1.0,1.0,0.354167,0.400,NaN,0.377083,RTV,Male,Male Headed,vsf
3,2023-08-04 18:59:13.000,2023-08-04 14:22:55.000,2023-08-04 15:24:06.000,2308030036,3637,6.0,6.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_b73b292d-66cd-4c9f-8a9a-4bcef8f8b03c.csv?uuid=uuid%3Ab73b292d-66cd-4c9f-8a9a-4bcef8f8b03c,2,28,...,1.0,1.0,0.701389,0.275,NaN,0.488194,RTV,Female,Female Headed,vsf
4,2023-08-05 19:30:19.000,2023-08-05 08:59:22.000,2023-08-05 10:09:25.000,2308030036,3970,31.0,31.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_aba161d1-ed84-4cf7-9633-c7db62e4f20d.csv?uuid=uuid%3Aaba161d1-ed84-4cf7-9633-c7db62e4f20d,2,37,...,1.0,1.0,0.812500,0.900,NaN,0.856250,RTV,Male,Male Headed,vsf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,2023-08-08 18:57:16.042,2023-08-08 11:45:06.019,2023-08-08 13:08:11.040,2308080450,4984,4.0,4.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_005188bd-d28e-41ab-8b8c-ea047c4280d2.csv?uuid=uuid%3A005188bd-d28e-41ab-8b8c-ea047c4280d2,7,111,...,1.0,1.0,1.000000,1.000,NaN,1.000000,RTV,Female,Female Headed,non vsf
1577,2023-08-11 17:20:48.970,2023-08-11 14:56:51.994,2023-08-11 16:15:06.970,2308100803,3952,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_a1312a8a-066c-48b2-a952-06b4fff4b133.csv?uuid=uuid%3Aa1312a8a-066c-48b2-a952-06b4fff4b133,7,111,...,1.0,1.0,1.000000,1.000,NaN,1.000000,RTV,Male,Male Headed,non vsf
1578,2023-08-10 21:03:41.011,2023-08-10 14:05:18.010,2023-08-10 14:50:40.992,2308100803,2723,3.0,3.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_eb46cfed-22aa-47dc-afbd-58764c77843f.csv?uuid=uuid%3Aeb46cfed-22aa-47dc-afbd-58764c77843f,7,106,...,1.0,1.0,0.750000,0.400,NaN,0.575000,RTV,Female,Female Headed,non vsf
1579,2023-08-13 15:52:01.027,2023-08-12 10:16:07.018,2023-08-13 15:51:30.010,2308112000,4274,9.0,9.0,https://rtvuganda.surveycto.com/view/submission-attachment/TA_b528262b-0af4-4ae5-822a-a6da9623e9c7.csv?uuid=uuid%3Ab528262b-0af4-4ae5-822a-a6da9623e9c7,7,118,...,NaN,NaN,1.000000,0.800,NaN,0.900000,RTV,Male,Male Headed,non vsf


## Loan Types

In [105]:
(rtv_vsf.groupby('state')['bank_loan_type'].value_counts(normalize=True) * 100).unstack().transpose()

state,non vsf,vsf
bank_loan_type,,
1.0,47.368421,100.0
2.0,52.631579,NaN


### Aggregating by Gender

In [106]:
non_vsf['Gender'].value_counts()

Gender
Male      1006
Female     353
Name: count, dtype: int64

In [107]:
adjusted_cpi = 80.66
rate = 3600

In [108]:
# Total Savings
((non_vsf.groupby(['Household_Head_Gender'])['Total Savings (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    60.118132
Male Headed      79.820095
Youth Headed     75.434234
Name: Total Savings (Ugx), dtype: float64

In [109]:
# Total Savings
((vsf.groupby(['Household_Head_Gender'])['Total Savings (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    79.637747
Male Headed      97.454326
Youth Headed     81.332167
Name: Total Savings (Ugx), dtype: float64

In [110]:
for frame in [vsf, non_vsf]:
    percent = (len(frame[frame['Total Savings (Ugx)'] != 0]) / len(frame))
    print(f"{get_df_name(frame)}:{percent}")

vsf:1.0
non_vsf:0.9882266372332598


In [111]:
vsf.groupby('Household_Head_Gender').apply(lambda x: len(x[x['Total Savings (Ugx)'] != 0]) / len(x))

Household_Head_Gender
Female Headed    1.0
Male Headed      1.0
Youth Headed     1.0
dtype: float64

In [112]:
non_vsf.groupby('Household_Head_Gender').apply(lambda x: len(x[x['Total Savings (Ugx)'] != 0]) / len(x))

Household_Head_Gender
Female Headed    0.988406
Male Headed      0.988372
Youth Headed     0.985294
dtype: float64

In [113]:
## Loan Amount for only those that saved
((subset.groupby(['state','Household_Head_Gender'])['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate).unstack()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
state,,,
non vsf,54.069256,109.103818,38.985667
vsf,63.108981,99.049465,44.811111


In [114]:
## Loan Amount for only those that saved
((vsf.groupby(['Household_Head_Gender'])['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    17.749401
Male Headed      35.470417
Youth Headed      8.962222
Name: Loan Amount (Ugx), dtype: float64

In [115]:
## Loan Amount for only those that saved
((non_vsf.groupby(['Household_Head_Gender'])['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    16.612583
Male Headed      36.790822
Youth Headed     10.893054
Name: Loan Amount (Ugx), dtype: float64

In [116]:
for frame in [vsf, non_vsf]:
    for gender in frame['Household_Head_Gender'].unique():
        sub_frame = frame[frame['Household_Head_Gender'] == gender]
        debt = sub_frame[columns].sum(axis=1).mean() * (adjusted_cpi / 100) / rate
        print(f"{get_df_name(frame)} ({gender}):\t{debt}")

vsf (Female Headed):	14.411323350694444
vsf (Male Headed):	26.349993055555558
vsf (Youth Headed):	6.721666666666667
non_vsf (Male Headed):	29.57245874318769
non_vsf (Female Headed):	12.754094282769728
non_vsf (Youth Headed):	12.336235294117646


In [117]:
(vsf.groupby(['Household_Head_Gender'])['borrowed_past_12_months'].value_counts(normalize=True)).unstack().transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
borrowed_past_12_months,,,
1,1.0,1.0,1.0


In [118]:
(non_vsf.groupby(['Household_Head_Gender'])['borrowed_past_12_months'].value_counts(normalize=True)).unstack().transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
borrowed_past_12_months,,,
0,0.692754,0.662791,0.720588
1,0.307246,0.337209,0.279412


In [119]:
for frame in [vsf, non_vsf]:
    for gender in frame['Household_Head_Gender'].unique():
        sub_frame = frame[(frame['Household_Head_Gender'] == gender) & (frame['Interest'] != 0.)]
        interest = sub_frame['Interest'].mean()
        print(f"{get_df_name(frame)} (Gender: {gender}):\t{interest}")

vsf (Gender: Female Headed):	2.1422222222222222
vsf (Gender: Male Headed):	2.285849056603774
vsf (Gender: Youth Headed):	2.5
non_vsf (Gender: Male Headed):	4.151792452830189
non_vsf (Gender: Female Headed):	4.334615384615384
non_vsf (Gender: Youth Headed):	4.24


In [120]:
group_int = df_vsla[df_vsla['Interest'] != 0.]
group_int.groupby('Household_Head_Gender')['Interest'].mean()

Household_Head_Gender
Female Headed    4.011148
Male Headed      3.885229
Youth Headed     4.074286
Name: Interest, dtype: float64

In [121]:
((vsf.groupby(['Household_Head_Gender'])['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
purpose_for_borrow,,,
1.0,0.222222,0.245283,NaN
3.0,0.055556,0.207547,0.5
4.0,0.444444,0.264151,NaN
6.0,0.277778,0.188679,0.5
7.0,NaN,0.037736,NaN
8.0,NaN,0.056604,NaN


In [122]:
((non_vsf.groupby(['Household_Head_Gender'])['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
purpose_for_borrow,,,
1.0,0.179245,0.278997,0.210526
2.0,0.037736,0.043887,0.052632
3.0,0.103774,0.131661,0.157895
4.0,0.235849,0.197492,0.105263
5.0,0.047170,0.031348,NaN
6.0,0.264151,0.206897,0.368421
7.0,0.018868,0.034483,0.052632
8.0,0.084906,0.040752,NaN
9.0,0.018868,0.018809,0.052632


In [123]:
(vsf.groupby('Household_Head_Gender')[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
Personal Business & Self Employment (USD_Cons_rate),172.921877,169.951437,237.722944
Seasonal Crops Income (USD_Cons_rate),191.309136,188.763324,159.762814
Seasonal Agriculture Value (USD_Cons_rate),439.128059,439.876358,331.362483
Livestock Income (USD_Cons_rate),40.193466,35.270583,10.418583
Assets (USD_Cons_rate),233.623428,240.827174,278.949167
HH Income (USD_Cons_rate),637.812648,642.810393,593.937669
HH Income + Consumption + Residues (USD_Cons_rate),862.502028,906.689326,629.606356
HH Income/Day (USD_Cons_rate),1.747432,1.761124,1.627226


In [124]:
(non_vsf.groupby('Household_Head_Gender')[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
Personal Business & Self Employment (USD_Cons_rate),69.633934,91.750179,121.086858
Seasonal Crops Income (USD_Cons_rate),147.371676,147.371676,147.371676
Seasonal Agriculture Value (USD_Cons_rate),363.211986,318.933753,297.786143
Livestock Income (USD_Cons_rate),24.533759,30.200865,18.758063
Assets (USD_Cons_rate),227.444964,242.915420,182.753550
HH Income (USD_Cons_rate),472.445687,525.532657,519.274808
HH Income + Consumption + Residues (USD_Cons_rate),793.744309,848.893293,806.666396
HH Income/Day (USD_Cons_rate),1.294372,1.439815,1.422671


In [125]:
vsf.to_excel('final_vsf.xlsx', index=False)

In [126]:
vsf.shape

(222, 5666)

In [199]:
a = np.array([2,3,4,5])
b = np.array([100,350,300,150])

In [203]:
c = b/a
c.mean()

67.91666666666667

In [201]:
b.mean()/a.mean()

64.28571428571429